
# DALL-E 2 - Variações de Imagem

Gera variações a partir de uma imagem existente, mantendo a composição geral e explorando estilos parecidos.

In [1]:

import base64
import os
from pathlib import Path
from urllib.request import urlopen

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [2]:

variations = 3
image_path = Path('assets/lab_base.png')

if not image_path.exists():
    raise FileNotFoundError('Execute a célula anterior para gerar `assets/lab_base.png` antes de criar variações.')

with image_path.open('rb') as image_file:
    variation = client.images.create_variation(
        model='dall-e-2',
        image=image_file,
        n=variations,
        size='1024x1024',
        response_format='b64_json'
    )

outputs_dir = Path('outputs')
outputs_dir.mkdir(parents=True, exist_ok=True)

for idx, data in enumerate(variation.data, start=1):
    image_base64 = getattr(data, 'b64_json', None)

    if image_base64:
        image_bytes = base64.b64decode(image_base64)
    else:
        image_url = getattr(data, 'url', None)
        if image_url is None:
            raise RuntimeError('A resposta não possui b64_json nem url; revise o payload retornado.')
        with urlopen(image_url) as response_stream:
            image_bytes = response_stream.read()

    filename = outputs_dir / f'dalle_variacao_{idx}.png'
    filename.write_bytes(image_bytes)
    print(f'Variação {idx} salva em {filename.resolve()}')


Variação 1 salva em /home/lucas-biason/Projetos/Estudos/Projetos de Portifolio/ia-ml-knowledge-base/knowledge-base/04 OpenAI API Recipes/outputs/dalle_variacao_1.png
Variação 2 salva em /home/lucas-biason/Projetos/Estudos/Projetos de Portifolio/ia-ml-knowledge-base/knowledge-base/04 OpenAI API Recipes/outputs/dalle_variacao_2.png
Variação 3 salva em /home/lucas-biason/Projetos/Estudos/Projetos de Portifolio/ia-ml-knowledge-base/knowledge-base/04 OpenAI API Recipes/outputs/dalle_variacao_3.png



## Observações

O método images.create_variation da API atual não aceita um prompt extra para descrever alterações desejadas. Ele apenas recebe a imagem base (e, opcionalmente, uma máscara). A variação que o modelo faz é implícita: ele gera versões semelhantes, sem controle textual.

Se você precisa orientar as modificações com prompt (“deixar mais neon”, “adicionar drones”, etc.), há duas alternativas:
* Gerar uma nova imagem com images.generate, ajustando o prompt — efetivamente cria outra base com as mudanças.
* Usar images.edit com máscara — você fornece a imagem original, uma máscara que marca as áreas a serem alteradas e um prompt descrevendo como preencher essas regiões.